![tower bridge](london.jpg)

London, or as the Romans called it "Londonium"! Home to [over 8.5 million residents](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/populationandhouseholdestimatesenglandandwales/census2021unroundeddata#population-and-household-estimates-england-and-wales-data) who speak over [300 languages](https://web.archive.org/web/20080924084621/http://www.cilt.org.uk/faqs/langspoken.htm). While the City of London is a little over one square mile (hence its nickname "The Square Mile"), Greater London has grown to encompass 32 boroughs spanning a total area of 606 square miles! 

![underground train leaving a platform](tube.jpg)

Given the city's roads were originally designed for horse and cart, this area and population growth has required the development of an efficient public transport system! Since the year 2000, this has been through the local government body called **Transport for London**, or *TfL*, which is managed by the London Mayor's office. Their remit covers the London Underground, Overground, Docklands Light Railway (DLR), buses, trams, river services (clipper and [Emirates Airline cable car](https://en.wikipedia.org/wiki/London_cable_car)), roads, and even taxis.

The Mayor of London's office make their data available to the public [here](https://data.london.gov.uk/dataset). In this project, you will work with a slightly modified version of a dataset containing information about public transport journey volume by transport type. 

The data has been loaded into a **Snowflake** database called `TFL` with a single table called `JOURNEYS`, including the following data:

## TFL.JOURNEYS

| Column | Definition | Data type |
|--------|------------|-----------|
| `MONTH`| Month in number format, e.g., `1` equals January | `INTEGER` |
| `YEAR` | Year | `INTEGER` |
| `DAYS` | Number of days in the given month | `INTEGER` |
| `REPORT_DATE` | Date that the data was reported | `DATE` |
| `JOURNEY_TYPE` | Method of transport used | `VARCHAR` |
| `JOURNEYS_MILLIONS` | Millions of journeys, measured in decimals | `FLOAT` |

Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

You will execute SQL queries to answer three questions, as listed in the instructions.

In [1]:
# SQL Engine imports
from dotenv import load_dotenv
import os
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import warnings
warnings.filterwarnings("ignore")

# Python data analysis imports
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

Initialize SQL

In [2]:
load_dotenv()
user = os.environ.get("USER")
pw = os.environ.get("PASS")
db = os.environ.get("DB")
host = os.environ.get("HOST")
api = os.environ.get("API")
port = 5432

In [3]:
uri = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}"
alchemyEngine = create_engine(uri)
conn = alchemyEngine.connect()

Load data

In [4]:
df = pd.read_csv('tfl_journeys_final.csv')
df.to_sql('journeys', conn, if_exists='replace', index=False)

936

In [5]:
def query(stmt: str):
    """Executes a given SQL statement and returns a Pandas DataFrame given the results.
    
    Parameters
    ----------
    stmt: str
        The SQL statement to be executed
    """
    global conn
    result = pd.read_sql_query(stmt, conn)
    return result

Exploring the data

In [6]:
query('''SELECT * FROM JOURNEYS''')

,month,year,days,report_date,journey_type,journeys_millions
0,1,2010,31,06/30/2010,Underground & DLR,96.836391
1,2,2010,28,06/30/2010,Underground & DLR,90.330504
2,3,2010,31,06/30/2010,Underground & DLR,90.038014
3,4,2010,30,09/30/2010,Underground & DLR,92.544093
4,5,2010,31,09/30/2010,Underground & DLR,88.662911
...,...,...,...,...,...,...
931,8,2022,31,12/31/2022,TfL Rail,12.515850
932,9,2022,30,12/31/2022,TfL Rail,15.223201
933,10,2022,31,03/31/2023,TfL Rail,8.875033
934,11,2022,30,03/31/2023,TfL Rail,13.404840


## Questions:

1. What are the most popular transport types, measured by the total number of journeys? The output should contain two columns, 1) JOURNEY_TYPE and 2) TOTAL_JOURNEYS_MILLIONS, and be sorted by the second column in descending order. Save the query as most_popular_transport_types.

In [7]:
query('''
    -- most_popular_transport_types
    select JOURNEY_TYPE, SUM(JOURNEYS.JOURNEYS_MILLIONS) as TOTAL_JOURNEYS_MILLIONS
    from JOURNEYS
    group by JOURNEY_TYPE
    order by TOTAL_JOURNEYS_MILLIONS DESC
''')

,journey_type,total_journeys_millions
0,Bus,24905.193947
1,Underground & DLR,15020.466544
2,Overground,1666.845666
3,TfL Rail,411.313421
4,Tram,314.689875
5,Emirates Airline,14.583718


2. Which five months and years were the most popular for the Emirates Airline? Return an output containing MONTH, YEAR, and JOURNEYS_MILLIONS, with the latter rounded to two decimal places and aliased as ROUNDED_JOURNEYS_MILLIONS. Exclude null values and save the result as emirates_airline_popularity.

In [8]:
query('''
    -- emirates_airline_popularity
    select month, year, ROUND(sum(journeys_millions)::NUMERIC,2) as ROUNDED_JOURNEYS_MILLIONS
    from JOURNEYS
    where JOURNEYS.JOURNEY_TYPE = 'Emirates Airline'
    group by month, JOURNEYS.YEAR
    having sum(journeys_millions) is not null
    order by ROUNDED_JOURNEYS_MILLIONS desc limit 5
''')

,month,year,rounded_journeys_millions
0,5,2012,0.53
1,6,2012,0.38
2,4,2012,0.24
3,5,2015,0.19
4,5,2013,0.19


3. Find the five years with the lowest volume of Underground & DLR journeys, saving as least_popular_years_tube. The results should contain the columns YEAR, JOURNEY_TYPE, and TOTAL_JOURNEYS_MILLIONS.

In [9]:
query('''
    -- least_popular_years_tube
    select year, JOURNEYS.JOURNEY_TYPE, sum(JOURNEYS.JOURNEYS_MILLIONS) as TOTAL_JOURNEYS_MILLIONS
    from JOURNEYS
    where JOURNEYS.JOURNEY_TYPE = 'Underground & DLR'
    group by JOURNEYS.YEAR, JOURNEYS.JOURNEY_TYPE
    order by TOTAL_JOURNEYS_MILLIONS limit 5
''')

,year,journey_type,total_journeys_millions
0,2020,Underground & DLR,310.179316
1,2021,Underground & DLR,748.452544
2,2022,Underground & DLR,1064.859009
3,2010,Underground & DLR,1096.145588
4,2011,Underground & DLR,1156.647654
